# Raw Data Ingestion and Structuring

This notebook covers the initial steps of obtaining and preparing the data for analysis. It includes importing necessary libraries, mounting Google Drive to access files, importing custom utility functions, downloading the raw dataset from Kaggle, processing the raw data to create static and dynamic features, saving this processed data, and finally aggregating the static and dynamic features into a single DataFrame suitable for downstream tasks.

## Import Necessary Libraries

This section imports essential Python libraries such as `kagglehub` for dataset interaction, `os` and `sys` for system operations, and `pandas` and `numpy` for data manipulation and numerical computing.

In [1]:
# Reference: https://github.com/Kaggle/kagglehub/blob/main/README.md#installation
%pip install kagglehub

In [2]:
import kagglehub
import sys
import os
import pandas as pd
import numpy as np

## Mount Google Drive

This section mounts the Google Drive to access files directly within the Colab environment.

In [3]:
from google.colab import drive

# Mount Google drive
drive.mount('/content/drive')

# Base file path
basePath = 'drive/MyDrive/Colab Notebooks/AAI-590-01_02/AAI590_CapstoneProject'

Mounted at /content/drive


## Import Custom Modules

This section adds the source directory to the system path, allowing the notebook to import custom utility functions

In [4]:
# Reference: https://coderivers.org/blog/sys-path-append-python/

# Note: use below code if running in the local machine
# sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Note: use below code if running in the Google colab
sys.path.append(os.path.join(basePath, 'src'))

from utils import copy_entire_directory, convert_time_minutes

## Download Kaggle Dataset

This section uses the `kagglehub` library to log in to Kaggle and download the specified dataset.

In [5]:
# References:
# https://github.com/Kaggle/kagglehub/blob/main/README.md#option-1-calling-kagglehublogin
# https://github.com/Kaggle/kaggle-api/tree/main/docs#api-credentials
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [6]:
# References:
# https://www.kaggle.com/datasets/msafi04/predict-mortality-of-icu-patients-physionet
# https://github.com/Kaggle/kagglehub/blob/main/README.md#download-dataset

# Download latest version
path = kagglehub.dataset_download("msafi04/predict-mortality-of-icu-patients-physionet")

print("Path to dataset files:", path)

100%|██████████| 7.64M/7.64M [00:00<00:00, 106MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/msafi04/predict-mortality-of-icu-patients-physionet/versions/1


In [7]:
# Note: use below code if running in the local machine

# Copy dataset from cache to target directory
# target_dir = "../data/raw"
# copy_entire_directory(path, target_dir)

In [7]:
# Note: use below code if running in the Google colab

# Copy dataset from cache to target directory
target_dir = os.path.join(basePath, 'data', 'raw')
copy_entire_directory(path, target_dir)

## Save Structured Data

This section saves the processed static data, which was collected by iterating through patient files, extracting static features, merging with outcome data, and structuring it into a pandas DataFrame. It also saves the dynamic tensors, which were created by processing time-varying vital data for each patient, converting timestamps to minutes, pivoting the data to a time-by-feature format, and converting it into NumPy arrays (tensors). This allows the cleaned and structured data to be easily loaded for subsequent steps without reprocessing the raw files.

In [8]:
# Note: use below code if running in the local machine
# data_dir = r'../data/raw/set-a/set-a'
# outcomes_path = r'../data/raw/Outcomes-a.txt'

# Note: use below code if running in the Google colab
data_dir = os.path.join(target_dir, 'set-a', 'set-a')
outcomes_path = os.path.join(target_dir, 'Outcomes-a.txt')

# To collect unique dynamic and static features
all_dynamic_features = set()
all_static_features = set()

# Load outcome data and extract static feature names (excluding RecordID)
outcomes_df = pd.read_csv(outcomes_path)
outcomes_df.set_index('RecordID', inplace=True)
outcome_static_features = set(outcomes_df.columns) - {'RecordID'}

# Add static feature names from patient and outcome data into static set
static_params = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']
all_static_features.update(static_params)
all_static_features.update(outcome_static_features)

# Iterate through all patient files and collect all feature names
for file in os.listdir(data_dir):
    if file.endswith('.txt'):
        df = pd.read_csv(os.path.join(data_dir, file))
        all_params = df['Parameter']
        all_dynamic_features.update(all_params)

# Derive dynamic feature names by excluding all known static feature names
all_dynamic_features = all_dynamic_features - all_static_features

# Add 'RecordID' and 'Minutes' to the dynamic features set
all_dynamic_features.add('RecordID')
all_dynamic_features.add('Minutes')

# Convert to lists
all_dynamic_features = list(all_dynamic_features)
all_static_features = list(all_static_features)

# Reorder dynamic features with 'RecordID' and 'Minutes' at the beginning
dynamic_features_ordered = ['RecordID', 'Minutes']
dynamic_features_ordered.extend(sorted([f for f in all_dynamic_features if f not in ['RecordID', 'Minutes']]))
all_dynamic_features = dynamic_features_ordered

# Reorder static features with 'RecordID' at the beginning
static_features_ordered = ['RecordID']
static_features_ordered.extend(sorted([f for f in all_static_features if f not in ['RecordID']]))
all_static_features = static_features_ordered

# View static and dynamic feature names
print("Dynamic features:", all_dynamic_features)
print("Static features:", all_static_features)

Dynamic features: ['RecordID', 'Minutes', 'ALP', 'ALT', 'AST', 'Albumin', 'BUN', 'Bilirubin', 'Cholesterol', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT', 'HR', 'K', 'Lactate', 'MAP', 'MechVent', 'Mg', 'NIDiasABP', 'NIMAP', 'NISysABP', 'Na', 'PaCO2', 'PaO2', 'Platelets', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC', 'pH']
Static features: ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'In-hospital_death', 'Length_of_stay', 'SAPS-I', 'SOFA', 'Survival', 'Weight']


In [9]:
# Builds static and dynamic tensors
patient_static_data = []
patient_dynamic_tensors = []

# Iterate through all patient files
for file in os.listdir(data_dir):
    if file.endswith('.txt'):
        path = os.path.join(data_dir, file)
        df = pd.read_csv(path)

        # Get RecordID
        rid = int(df[df['Parameter'] == 'RecordID']['Value'].values[0])

        # Static Features: Filter by static feature set
        df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
        # Use the reordered all_static_features list for filtering
        static_subset = df[df['Parameter'].isin(all_static_features)]
        static_dict = static_subset.drop_duplicates('Parameter').set_index('Parameter')['Value'].to_dict()
        static_dict['RecordID'] = rid # update with integer value

        # Inject static data from outcomes
        if rid in outcomes_df.index:
            static_dict.update(outcomes_df.loc[rid].to_dict())

        # Reorder and fill missing static features with -1 using the reordered list
        ordered_static = {key: static_dict.get(key, -1) for key in all_static_features}
        patient_static_data.append(ordered_static)

        # Dynamic Features: Filter by dynamic feature set (excluding RecordID and Minutes for pivoting)
        dynamic_cols_for_pivot = [f for f in all_dynamic_features if f not in ['RecordID', 'Minutes']]
        dynamic_subset = df[df['Parameter'].isin(dynamic_cols_for_pivot)].copy()

        # Converts time string in 'HH:MM' format into total minutes since ICU admission
        dynamic_subset['Minutes'] = dynamic_subset['Time'].apply(convert_time_minutes)

        # Pivot the dynamic data
        pivot = dynamic_subset.pivot_table(index='Minutes', columns='Parameter', values='Value', aggfunc='last')

        # Add RecordID and Minutes columns after pivoting
        pivot['RecordID'] = rid
        pivot['Minutes'] = pivot.index

        # Reindex the columns to match the reordered all_dynamic_features, sort by index and fill NaN with -1
        pivot = pivot.reindex(columns=all_dynamic_features).sort_index().fillna(-1)

        # Convert the pivoted DataFrame to a NumPy array (tensor)
        tensor = pivot.to_numpy()
        # Append the dynamic tensor to the list of patient dynamic tensors
        patient_dynamic_tensors.append(tensor)

# Display data for the first patient
print("Static keys:", patient_static_data[0].keys())
print("Dynamic tensor shape (time × features):", patient_dynamic_tensors[0].shape)

Static keys: dict_keys(['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'In-hospital_death', 'Length_of_stay', 'SAPS-I', 'SOFA', 'Survival', 'Weight'])
Dynamic tensor shape (time × features): (59, 38)


In [10]:
# Convert patient static data tensor to a dataframe and display the first few rows
df_static = pd.DataFrame(patient_static_data)
df_static.head()

,RecordID,Age,Gender,Height,ICUType,In-hospital_death,Length_of_stay,SAPS-I,SOFA,Survival,Weight
0,137763,52.0,1.0,185.4,1.0,0,8,9,2,1130,90.0
1,137035,65.0,1.0,-1.0,2.0,1,14,11,3,13,-1.0
2,135275,47.0,0.0,-1.0,1.0,0,3,4,1,10,86.6
3,138905,35.0,0.0,154.9,2.0,0,6,-1,7,-1,67.0
4,141424,64.0,0.0,-1.0,3.0,1,14,-1,-1,14,74.3


In [11]:
# Display dynamic tensor shape for each patient
[t.shape for t in patient_dynamic_tensors]

[(59, 38),
 (54, 38),
 (58, 38),
 (96, 38),
 (78, 38),
 (80, 38),
 (66, 38),
 (101, 38),
 (62, 38),
 (82, 38),
 (73, 38),
 (113, 38),
 (76, 38),
 (65, 38),
 (74, 38),
 (75, 38),
 (47, 38),
 (74, 38),
 (62, 38),
 (94, 38),
 (77, 38),
 (70, 38),
 (90, 38),
 (57, 38),
 (99, 38),
 (62, 38),
 (115, 38),
 (55, 38),
 (49, 38),
 (74, 38),
 (62, 38),
 (77, 38),
 (112, 38),
 (72, 38),
 (109, 38),
 (11, 38),
 (112, 38),
 (77, 38),
 (61, 38),
 (13, 38),
 (86, 38),
 (98, 38),
 (100, 38),
 (56, 38),
 (64, 38),
 (55, 38),
 (52, 38),
 (86, 38),
 (37, 38),
 (91, 38),
 (67, 38),
 (70, 38),
 (83, 38),
 (71, 38),
 (38, 38),
 (77, 38),
 (97, 38),
 (60, 38),
 (91, 38),
 (81, 38),
 (82, 38),
 (55, 38),
 (35, 38),
 (6, 38),
 (85, 38),
 (22, 38),
 (81, 38),
 (73, 38),
 (77, 38),
 (51, 38),
 (99, 38),
 (91, 38),
 (86, 38),
 (80, 38),
 (111, 38),
 (60, 38),
 (92, 38),
 (69, 38),
 (88, 38),
 (78, 38),
 (83, 38),
 (85, 38),
 (60, 38),
 (53, 38),
 (89, 38),
 (136, 38),
 (77, 38),
 (119, 38),
 (10, 38),
 (53, 38),
 

In [12]:
# Create a list of DataFrames, one for each patient
patient_dataframes = [pd.DataFrame(patient_tensor, columns=all_dynamic_features) for patient_tensor in patient_dynamic_tensors]

# Concatenate all patient dataframes to a DataFrame
df_dynamic = pd.concat(patient_dataframes, ignore_index=True)

# Convert 'RecordID' and 'Minutes' columns to integer type
df_dynamic['RecordID'] = df_dynamic['RecordID'].astype(int)
df_dynamic['Minutes'] = df_dynamic['Minutes'].astype(int)

# Display first few rows
df_dynamic.head()

/tmp/ipython-input-12-2280472372.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_dynamic = pd.concat(patient_dataframes, ignore_index=True)


,RecordID,Minutes,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,...,Platelets,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,pH
0,137763,32,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,35.9,-1.0,-1.0,-1.0,-1.0,-1.0
1,137763,57,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,137763,62,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,137763,92,229.0,5.0,22.0,2.9,15.0,0.8,-1.0,0.5,...,112.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4.6,-1.0
4,137763,122,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [13]:
processed_dir = os.path.join(basePath, 'data', 'processed')

# Note: use below code if running in the local machine
# static_df_file = r'../data/processed/patient_static_data_df.csv'
# dynamic_df_file = r'../data/processed/patient_dynamic_tensors_df.csv'

# Note: use below code if running in the Google colab
static_df_file = os.path.join(processed_dir, 'patient_static_data_df.csv')
dynamic_df_file = os.path.join(processed_dir, 'patient_dynamic_tensors_df.csv')

# Save the static dataframe
df_static.to_csv(static_df_file, index=False)

# Save the dynamic dataframe
df_dynamic.to_csv(dynamic_df_file, index=False)

## Aggregate Patient Features

This section aggregates the processed static data and dynamic tensors for each patient into a single pandas DataFrame. It combines the static features directly and calculates summary statistics (mean, std, min, max, count) for each dynamic feature across time, creating a flattened representation of the data suitable for modeling.

In [14]:
# Combine static and dynamic data for each patient into a dictionary
patient_combined_data = []
for i in range(len(patient_static_data)):
    patient_combined_data.append({
        'static': patient_static_data[i],
        'dynamic': patient_dynamic_tensors[i]
    })

In [15]:
# Process each patient's combined data
patient_features_list = []

for patient_data_dict in patient_combined_data:
    features = {}

    # Extract static features, including RecordID
    static_data = patient_data_dict['static']
    features.update(static_data)

    # Calculate summary statistics for dynamic features
    dynamic_tensor = patient_data_dict['dynamic']
    dynamic_df = pd.DataFrame(dynamic_tensor, columns=all_dynamic_features)

    # Exclude 'RecordID' and 'Minutes' from aggregation
    dynamic_cols_for_aggregation = [col for col in dynamic_df.columns if col not in ['RecordID', 'Minutes']]

    for col in dynamic_cols_for_aggregation:
        # Exclude the -1 fill value from calculations
        valid_data = dynamic_df[col][dynamic_df[col] != -1]

        if not valid_data.empty:
            features[f'{col}_mean'] = valid_data.mean()
            features[f'{col}_std'] = valid_data.std()
            features[f'{col}_min'] = valid_data.min()
            features[f'{col}_max'] = valid_data.max()
            features[f'{col}_count'] = valid_data.count()
        else:
            # cases where all values are -1
            features[f'{col}_mean'] = np.nan
            features[f'{col}_std'] = np.nan
            features[f'{col}_min'] = np.nan
            features[f'{col}_max'] = np.nan
            features[f'{col}_count'] = 0

    patient_features_list.append(features)

# Create the aggregated dataframe
df_aggregated = pd.DataFrame(patient_features_list)

In [16]:
# Display the first few rows of the aggregated dataframe
df_aggregated.head()

,RecordID,Age,Gender,Height,ICUType,In-hospital_death,Length_of_stay,SAPS-I,SOFA,Survival,...,WBC_mean,WBC_std,WBC_min,WBC_max,WBC_count,pH_mean,pH_std,pH_min,pH_max,pH_count
0,137763,52.0,1.0,185.4,1.0,0,8,9,2,1130,...,5.733333,1.059874,4.6,6.7,3,NaN,NaN,NaN,NaN,0
1,137035,65.0,1.0,-1.0,2.0,1,14,11,3,13,...,1.687500,0.188509,1.5,2.0,8,NaN,NaN,NaN,NaN,0
2,135275,47.0,0.0,-1.0,1.0,0,3,4,1,10,...,8.433333,1.650253,6.8,10.1,3,NaN,NaN,NaN,NaN,0
3,138905,35.0,0.0,154.9,2.0,0,6,-1,7,-1,...,9.600000,NaN,9.6,9.6,1,7.374706,0.052929,7.27,7.44,17
4,141424,64.0,0.0,-1.0,3.0,1,14,-1,-1,14,...,13.933333,4.787344,10.5,21.4,6,7.430000,0.039158,7.39,7.48,4


In [17]:
# Note: use below code if running in the local machine
# df_aggregated_file = r'../data/processed/patient_aggregated_features_df.csv'

# Note: use below code if running in the Google colab
df_aggregated_file = os.path.join(processed_dir, 'patient_aggregated_features_df.csv')

# Save the aggregated dataframe
df_aggregated.to_csv(df_aggregated_file, index=False)